We have received a request from a user seeking to ascertain how many MA holders there are for metered dose inhalers and enviromentals inhalers. This is related to proposed wotk on phasing out MDIs due to the enviromental impact. We don't have a list of MAs, that should be sought from the MHRA but a very good approximation is what is in the NHS dm+d. 

In [1]:
##import libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [2]:
##we should be able to get this information from the NHS dmd
sql = '''
SELECT
  *
FROM
  `ebmdatalab.dmd2.vw__available_ampps_with_pricing`
WHERE
  vmp_formroute IN ('powderinhalation.inhalation', ## DPI
    'pressurizedinhalation.inhalation') ##MDI
  AND price != 0 ## this will exclude make believe products
ORDER BY
  price_valid_on DESC
  '''
df_inhalerma = bq.cached_read(sql, csv_path='df_inhalerma.csv', use_cache=True)  # add `use_cache=False` to override

df_inhalerma.head()

,vmpp_id,ampp_id,ampp_bnf_code,vmpp_bnf_code,ampp_name,vmpp_name,dt_price,price,price_valid_on,price_prev,price_basis,licensed_route,vmp_formroute,supplier,gtin,gtin_end_date,possible_wholesaler,category,cat_a_weighting
0,1229311000001105,34215611000001102,0302000N0BGACBG,0302000N0AABGBG,AirFluSal 25micrograms/dose / 250micrograms/do...,Fluticasone 250micrograms/dose / Salmeterol 25...,2932,2495,2019-01-16,2995.0,NHS Indicative Price,Inhalation,pressurizedinhalation.inhalation,Sandoz Ltd,5.050650e+12,NaN,False,Part VIIIA Category C,NaN
1,1229311000001105,34675911000001101,0302000N0BJABBG,0302000N0AABGBG,Aloflute 25micrograms/dose / 250micrograms/dos...,Fluticasone 250micrograms/dose / Salmeterol 25...,2932,2832,2019-01-03,3885.0,NHS Indicative Price,Inhalation,pressurizedinhalation.inhalation,Mylan,5.016695e+12,NaN,False,Part VIIIA Category C,NaN
2,1282011000001105,34023711000001105,0302000N0BIAABF,0302000N0AABFBF,Sereflo 25micrograms/dose / 125micrograms/dose...,Fluticasone 125micrograms/dose / Salmeterol 25...,2345,1499,2019-01-02,1649.0,NHS Indicative Price,Inhalation,pressurizedinhalation.inhalation,Cipla EU Ltd,5.015201e+12,NaN,False,Part VIIIA Category C,NaN
3,3176611000001101,35430211000001106,0302000C0BSAABE,0302000C0AABEBE,Kelhale 50micrograms/dose inhaler (Cipla EU Lt...,Beclometasone 50micrograms/dose inhaler CFC fr...,370,520,2019-01-02,606.0,NHS Indicative Price,Inhalation,pressurizedinhalation.inhalation,Cipla EU Ltd,8.901117e+12,NaN,False,Part VIIIA Category C,NaN
4,3175511000001107,35430411000001105,0302000C0BSABBF,0302000C0AABFBF,Kelhale 100micrograms/dose inhaler (Cipla EU L...,Beclometasone 100micrograms/dose inhaler CFC f...,742,520,2019-01-02,1325.0,NHS Indicative Price,Inhalation,pressurizedinhalation.inhalation,Cipla EU Ltd,8.901117e+12,NaN,False,Part VIIIA Category C,NaN


In [3]:
df_inhalerma.count()

vmpp_id                115
ampp_id                115
ampp_bnf_code          110
vmpp_bnf_code          115
ampp_name              115
vmpp_name              115
dt_price               115
price                  115
price_valid_on          77
price_prev              77
price_basis            115
licensed_route         115
vmp_formroute          115
supplier               115
gtin                   114
gtin_end_date           14
possible_wholesaler    115
category               115
cat_a_weighting         15
dtype: int64

This indicates that there are 115 MDI/DPI in the NHS dmd

In [4]:
##Here we look for MA holders by counting unique suppliers
df_inhalerma.supplier.unique()



array(['Sandoz Ltd', 'Mylan', 'Cipla EU Ltd', 'Aspire Pharma Ltd',
       'GlaxoSmithKline UK Ltd', 'AstraZeneca UK Ltd',
       'Orion Pharma (UK) Ltd', 'Teva UK Ltd',
       'Novartis Pharmaceuticals UK Ltd', 'Boehringer Ingelheim Ltd',
       'Napp Pharmaceuticals Ltd', 'Sanofi', 'Chiesi Ltd',
       'Merck Sharp & Dohme Ltd', 'Meda Pharmaceuticals Ltd',
       'Glenmark Pharmaceuticals Europe Ltd', 'Kent Pharmaceuticals Ltd',
       'Alissa Healthcare Research Ltd'], dtype=object)

There are 18 unique suppliers with a price in dm+d. We have excluded those without a price to ensure "make believe" wholesaler products are not counted. There is a possibility that there may be a true MA holder in there. However 18 is a reasonable figure to give.

A further request has asked for
- a.	can you supply supplier name by individual product BNF codes?
or
- b.	If we can provide BNF codes for which inhalers are Metred Dose Inhalers, which are Dry Powder Inhalers etc is it possible to calculate the volume of each type sold by each supplier?


- a) this is in the csv above

- b) Supplier is at AMP(P) level we cannot do it accurately. Where it is prescribed by brand we will know market share but where it is prescribed at VMP level it'll be difficult.

In [5]:
sql = '''
SELECT
  p.bnf_code,
  p.bnf_name,
  dmd.form_route,
  SUM(items) AS total_items,
  SUM(actual_cost) AS total_cost,
  SUM(quantity) AS total_qty
FROM
  `ebmdatalab.hscic.prescribing_2019_03` AS p
LEFT JOIN
  ebmdatalab.dmd.form_dose AS dmd
ON
  p.bnf_code=dmd.bnf_code
WHERE
  p.bnf_code LIKE "030%"
  AND (form_route="powderinhalation.inhalation"
    OR form_route= "pressurizedinhalation.inhalation")
GROUP BY
  p.bnf_code,
  p.bnf_name,
  dmd.form_route
  '''
df2_inhalerma = bq.cached_read(sql, csv_path='df2_inhalerma.csv', use_cache=True)  # add `use_cache=False` to override

df2_inhalerma.head()

,bnf_code,bnf_name,form_route,total_items,total_cost,total_qty
0,0302000V0BCAAA0,Trelegy Ellipta_Inha 92/55/22mcg (30 D),powderinhalation.inhalation,121434,5.612027e+06,135708
1,0302000K0AAAUAU,Budesonide/Formoterol_InhaB/A 400/12(60D,powderinhalation.inhalation,16690,6.213466e+05,23878
2,0302000N0BDACBL,Flutiform_Inha 50/5mcg (120 D),pressurizedinhalation.inhalation,14098,2.345238e+05,17518
3,0301011U0BBAFAH,Serevent_Evohaler 25mcg (120 D),pressurizedinhalation.inhalation,5153,1.726249e+05,6348
4,0302000N0AABFBF,Fluticasone/Salmeterol_Inh 125/25mcg120D,pressurizedinhalation.inhalation,26752,7.358119e+05,33760


In [6]:
totalinh = df2_inhalerma['total_qty'].sum()
totalinh

17589318

In [7]:
df_amp_items = pd.merge(df_inhalerma, df2_inhalerma, left_on='ampp_bnf_code', right_on='bnf_code' )
df_amp_items.head(2)

,vmpp_id,ampp_id,ampp_bnf_code,vmpp_bnf_code,ampp_name,vmpp_name,dt_price,price,price_valid_on,price_prev,...,gtin_end_date,possible_wholesaler,category,cat_a_weighting,bnf_code,bnf_name,form_route,total_items,total_cost,total_qty
0,1229311000001105,34215611000001102,0302000N0BGACBG,0302000N0AABGBG,AirFluSal 25micrograms/dose / 250micrograms/do...,Fluticasone 250micrograms/dose / Salmeterol 25...,2932,2495,2019-01-16,2995.0,...,NaN,False,Part VIIIA Category C,NaN,0302000N0BGACBG,AirFluSal_Inh 250mcg/25mcg (120D),pressurizedinhalation.inhalation,8636,240851.01774,10386
1,1229311000001105,34675911000001101,0302000N0BJABBG,0302000N0AABGBG,Aloflute 25micrograms/dose / 250micrograms/dos...,Fluticasone 250micrograms/dose / Salmeterol 25...,2932,2832,2019-01-03,3885.0,...,NaN,False,Part VIIIA Category C,NaN,0302000N0BJABBG,Aloflute_Inh 250mcg/25mcg (120D),pressurizedinhalation.inhalation,242,7580.51232,288


In [8]:
totalinh_with_ampp = df_amp_items['total_qty'].sum()
totalinh_with_ampp

13679854

In [9]:
df_amp_items2 = df_amp_items.drop(['vmpp_id', 'ampp_id', 'dt_price', 'price', 'price_valid_on', 'price_prev', 'price_basis', 'licensed_route', 'gtin', 'gtin_end_date', 'possible_wholesaler', 'category', 'cat_a_weighting', 'vmp_formroute', 'bnf_name'], 'columns')

In [10]:
df_amp_items2.head(2)

,ampp_bnf_code,vmpp_bnf_code,ampp_name,vmpp_name,supplier,bnf_code,form_route,total_items,total_cost,total_qty
0,0302000N0BGACBG,0302000N0AABGBG,AirFluSal 25micrograms/dose / 250micrograms/do...,Fluticasone 250micrograms/dose / Salmeterol 25...,Sandoz Ltd,0302000N0BGACBG,pressurizedinhalation.inhalation,8636,240851.01774,10386
1,0302000N0BJABBG,0302000N0AABGBG,Aloflute 25micrograms/dose / 250micrograms/dos...,Fluticasone 250micrograms/dose / Salmeterol 25...,Mylan,0302000N0BJABBG,pressurizedinhalation.inhalation,242,7580.51232,288


In [11]:
suppqty = df_amp_items2.groupby(['supplier'])['total_qty'].sum()
suppqty


supplier
Alissa Healthcare Research Ltd           5
AstraZeneca UK Ltd                  502618
Boehringer Ingelheim Ltd           1001175
Chiesi Ltd                         1424114
Cipla EU Ltd                         23634
GlaxoSmithKline UK Ltd             1520460
Kent Pharmaceuticals Ltd              2994
Meda Pharmaceuticals Ltd               125
Merck Sharp & Dohme Ltd                 67
Mylan                               117226
Napp Pharmaceuticals Ltd             67362
Novartis Pharmaceuticals UK Ltd    3834710
Orion Pharma (UK) Ltd                45444
Sandoz Ltd                           34082
Sanofi                                 353
Teva UK Ltd                        5105485
Name: total_qty, dtype: int64